In [1]:
import os

In [2]:
%pwd

'c:\\Users\\PASCAL\\flight_price_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\PASCAL\\flight_price_prediction'

In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    metric_file_name: Path

In [11]:
from src.flightprice.constants import *
from src.flightprice.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelEvaluationConfig(
        root_dir=config.root_dir,
        data_path=config.data_path,
        model_path=config.model_path,
        metric_file_name=config.metric_file_name
    )

        return model_trainer_config

In [13]:
import os
import sys
from dataclasses import dataclass

from catboost import CatBoostRegressor
from sklearn.ensemble import (
    AdaBoostRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from src.flightprice.logger import logging
from src.flightprice.utils.common import evaluate_models
from src.flightprice.components.data_transformation import DataTransformation
import pickle

In [17]:
import os
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import logging

class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def evaluate_model(self, true, predicted):
        mae = mean_absolute_error(true, predicted)
        mse = mean_squared_error(true, predicted)
        rmse = np.sqrt(mse)
        r2_square = r2_score(true, predicted)
        return mae, rmse, r2_square

    def save_evaluation_results(self, model_list, r2_list):
        try:
            output_dir = os.path.join(self.config.root_dir, "model_evaluation")
            os.makedirs(output_dir, exist_ok=True)

            output_file = os.path.join(output_dir, "evaluation_results.txt")
            with open(output_file, "w") as file:
                for model, r2 in zip(model_list, r2_list):
                    file.write(f"{model}: R2 Score - {r2}\n")

        except Exception as e:
            logging.error(f"Error in save_evaluation_results: {str(e)}")


In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.evaluate_model()
except Exception as e:
    raise e

[2023-07-23 19:36:05,652: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-23 19:36:05,654: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-23 19:36:05,656: INFO: common: created directory at: artifacts]
[2023-07-23 19:36:05,657: INFO: common: created directory at: artifacts/model_trainer]


BoxKeyError: "'ConfigBox' object has no attribute 'metric_file_name'"